## **Instalação das Dependências Necessárias**
 **Install Required Dependencies**


In [ ]:
!pip install geemap pycrs geopandas folium rasterio -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.3 MB/s eta 0:00:00


 ## **Importação das Bibliotecas**
 **Import Libraries**

In [ ]:
# Importação de bibliotecas para análise geoespacial e manipulação de dados
# Importing libraries for geospatial analysis and data manipulation

import geemap        # Para visualização e análise de dados do Google Earth Engine
                     # For visualization and analysis of Google Earth Engine data
import ee            # Para conexão e manipulação de dados do Google Earth Engine
                     # For connecting to and manipulating Google Earth Engine data
import os            # Para operações de sistema e manipulação de arquivos
                     # For system operations and file manipulation
import numpy as np   # Para operações numéricas e manipulação de matrizes
                     # For numerical operations and matrix manipulation
import rasterio      # Para leitura e escrita de dados raster (imagens geoespaciais)
                     # For reading and writing raster data (geospatial images)
from osgeo import gdal, ogr, osr  # Para operações avançadas com dados geoespaciais
                                  # For advanced operations with geospatial data
import folium        # Para visualização interativa de mapas
                     # For interactive map visualization

## **Autenticação e Conexão com o Google Earth Engine**
 **Authenticate and Connect to Google Earth Engine**

In [ ]:
ee.Authenticate()  # Realiza a autenticação do usuário no Google Earth Engine
                   # Authenticates the user in Google Earth Engine

ee.Initialize(project='ee-yourproject')  # Inicializa a sessão com um projeto específico
                                           # Initializes the session with a specific project


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


## **Montar Google Drive no Ambiente de Trabalho**
**Mount Google Drive in the Working Environment**

In [ ]:
from google.colab import drive  # Importa a função para montar o Google Drive no Colab
                                # Imports the function to mount Google Drive in Colab

drive.mount('/content/drive')   # Monta o Google Drive no diretório especificado para acessar arquivos
                                # Mounts Google Drive to the specified directory to access files

# Agora os arquivos no Google Drive podem ser acessados a partir do caminho '/content/drive/MyDrive'
# Now files in Google Drive can be accessed from the path '/content/drive/MyDrive'


Mounted at /content/drive


# **NDVI**

## **Creating a map**


In [ ]:
# Centra o mapa nas coordenadas dos Açores com um nível de zoom de 12.
# Center the map on the Azores coordinates with a zoom level of 12.
my_map = geemap.Map(center=[37.79, -25.51], zoom=12, layer_ctrl=True)

# Adiciona uma camada de imagem de satélite como o mapa base.
# Adds a satellite imagery layer as the basemap.
my_map.add_basemap('SATELLITE')

# Importa o shapefile "areas.shp" do Google Drive e o converte para um objeto Earth Engine.
# Imports the "areas.shp" shapefile from Google Drive and converts it into an Earth Engine object.
#areas = geemap.shp_to_ee(r"/content/drive/MyDrive/areas.shp")

# Adiciona o shapefile ao mapa com a cor vermelha e o nome 'Areas'.
# Adds the shapefile to the map in red color with the name 'Areas'.
#my_map.addLayer(areas, {'color': 'red'}, name='Areas')

# Exibe o mapa resultante.
# Displays the resulting map.
my_map


Map(center=[37.79, -25.51], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…

## **Usando área desenhada no mapa**
 **Using a map draw area**

In [ ]:
# Define a área de interesse (AOI) com base nas funcionalidades desenhadas no mapa.
# Define the area of interest (AOI) based on features drawn on the map.
aoi = ee.FeatureCollection(my_map.draw_features)

## **Processando a coleção**
**Processing collection**

In [ ]:
# Cria o objeto de datas de início e término do período de análise.
# Creates the start and end date objects for the analysis period.
startdate = '2024-09-01'
enddate = '2024-09-17'

# Carrega os dados Sentinel-2, filtrando pela data e pela área de interesse (AOI).
# Load Sentinel-2 data, filtering by date and area of interest (AOI).
dataset = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')\
            .filterDate(startdate, enddate)\
            .filterBounds(aoi).first()

# Calcula o índice de vegetação por diferença normalizada (NDVI).
# Calculate the Normalized Difference Vegetation Index (NDVI).
ndvi = dataset.normalizedDifference(['B8', 'B4']).rename('NDVI')

# Define os parâmetros de visualização para o NDVI.
# Set visualization parameters for NDVI.
visParams = {
    'min': -1,
    'max': 0.5,
    'palette': [
        '#d73027', '#fee08b', '#d9ef8b', '#a6d96a', '#0e8d05'
    ],
}

# Coordenadas para Açores, Portugal.
# Coordinates for Azores, Portugal.
odemira_coords = [37.79, -25.51]

# Cria um objeto de mapa geemap centrado em Odemira, Portugal.
# Create a geemap object centered on Odemira, Portugal.
my_map = geemap.Map(center=odemira_coords, zoom=10)

# Adiciona o NDVI como uma camada ao mapa com os parâmetros de visualização definidos.
# Add NDVI as a layer to the map with the defined visualization parameters.
my_map.addLayer(ndvi, visParams, 'NDVI')

# Adiciona um painel de controle de camadas ao mapa.
# Add a layer control panel to the map.
my_map.addLayerControl()

# Exibe o mapa.
# Display the map.
my_map


Map(center=[37.79, -25.51], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…

## **Criando a url da camada XYZ**
**Creating a XYZ url**

In [ ]:
# Gera os parâmetros de visualização para a imagem NDVI, criando um dicionário com as informações de visualização.
# Generates visualization parameters for the NDVI image, creating a dictionary with display information.
map_id_dict = ndvi.getMapId(visParams)

# Extrai a URL dos tiles (azulejos) para a imagem NDVI, que será usada para renderizar a imagem no mapa.
# Extracts the tile URL for the NDVI image, which will be used to render the image on the map.
tile_url = map_id_dict['tile_fetcher'].url_format

# Imprime a URL dos tiles para visualização.
# Prints the tile URL for visualization.
print(f"URL dos tiles: {tile_url}")


URL dos tiles: https://earthengine.googleapis.com/v1/projects/ee-jessicaguchoa1/maps/4d4ee39af8d3934de8165fc800c23af9-3354e0798efe81e8e7ec25fbf510dfab/tiles/{z}/{x}/{y}


# Passo a passo para visualizar os dados no QGIS usando a URL XYZ gerada:



## Passo 1: Copiar a URL do Tile XYZ
 - No final da execução do código, uma URL XYZ será gerada para visualização dos tiles.
 - Copie a URL gerada, que será similar a:
   https://earthengine.googleapis.com/map/your_map_id/tiles/{z}/{x}/{y}
   (Substitua "your_map_id" pelo identificador gerado no código).

## Passo 2: Abrir o QGIS
 - Abra o QGIS no seu computador.

## Passo 3: Adicionar uma Nova Camada XYZ
 1. No painel "Camadas" (Layers), clique com o botão direito em "XYZ Tiles" (Azulejos XYZ).
 2. Selecione "Nova conexão" (New Connection).

## Passo 4: Configurar a Conexão XYZ
 1. Na janela de configuração:
    - Nome: Defina um nome para a camada, como `NDVI_GoogleEarthEngine`.
    - URL: Cole a URL XYZ copiada.
 2. Clique em OK para salvar a conexão.

## Passo 5: Adicionar a Camada XYZ ao Projeto
 - No painel "Azulejos XYZ" (XYZ Tiles), clique duas vezes na nova camada ou arraste-a para o mapa para visualizá-la.



# Step-by-step instructions to visualize data in QGIS using the generated XYZ URL:



## Step 1: Copy the XYZ Tile URL
 - At the end of the code execution, an XYZ URL for tile visualization will be generated.
 - Copy the generated URL, which will be similar to:
   https://earthengine.googleapis.com/map/your_map_id/tiles/{z}/{x}/{y}
   (Replace "your_map_id" with the identifier generated in the code).

## Step 2: Open QGIS
 - Launch QGIS on your computer.

## Step 3: Add a New XYZ Layer
 1. In the "Layers" panel, right-click on "XYZ Tiles."
 2. Select "New Connection."

## Step 4: Configure the XYZ Connection
 1. In the configuration window:
    - Name: Give the layer a name, such as `NDVI_GoogleEarthEngine`.
   - URL: Paste the copied XYZ URL.
 2. Click OK to save the connection.

## Step 5: Add the XYZ Layer to the Project
 - In the "XYZ Tiles" panel, double-click the new layer or drag it onto the map to display it.
